In [2]:
import pandas as pd
from myfun.addcol import addcolumns
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score
import joblib
import os

In [ ]:
df1 = pd.read_csv(f"./train_combine_groupby/01_traingroupby.csv")
df1 = addcolumns(df1)
def combine_weather(df1:pd.DataFrame):
    weather_df = pd.read_csv("../資料前處理及第三方資料/10mins_weather_data.csv")
    weather_df["DateTime"] =  pd.to_datetime(weather_df["DateTime"])
    weather_df["日期"] = pd.to_datetime(weather_df["DateTime"].dt.date)
    weather_df["時間"] = weather_df["DateTime"].dt.hour.astype(str).str.zfill(2)
    weather_df["時段"] = weather_df["DateTime"].dt.minute.astype(str).str.zfill(2)
    
    weather_df.fillna(0,inplace=True)
    df = pd.merge(
        df1,
        weather_df,
        left_on=["日期","時間","時段"],
        right_on=["日期","時間","時段"],
        how="left",
        suffixes=["","_r"]
    )

    return(df)
comdf = combine_weather(df1)
comdf.isna().sum()
comdf.loc[comdf["UVIndex"].isna(),["日期","時間","時段"]]

,日期,時間,時段
337,2024-01-06,06,20
338,2024-01-06,06,30
339,2024-01-06,06,40
340,2024-01-06,06,50
341,2024-01-06,07,00
...,...,...,...
10268,2024-07-14,18,20
10269,2024-07-14,18,30
10270,2024-07-14,18,40
10271,2024-07-14,18,50


In [6]:
def selecttime(df:pd.DataFrame):
    df["TimeGroup"] = pd.to_datetime(df["TimeGroup"])
    df = df[(df["TimeGroup"].dt.hour >= 9) & (df["TimeGroup"].dt.hour < 17)| ((df["TimeGroup"].dt.hour == 17) & (df["TimeGroup"].dt.minute == 0))]
    return(df)
selecttime(combine_weather(df1))

,Serial,LocationCode,TimeGroup,WindSpeed(m/s),Pressure(hpa),Temperature(°C),Humidity(%),Sunlight(Lux),Power(mW),日期,...,SunshineDuration,UVIndex,AirTemperature,DailyExtremeHighAirTemperature,DailyExtremeLowAirTemperature,RelativeHumidity,VisibilityDescription,StationLatitude,StationLongitude,StationAltitude
15,20240101090001,1,2024-01-01 09:00:00,1.91588,1017.906000,18.33,92.29,8395.250000,17.368000,2024-01-01,...,0.0,0.0,18.8,19.3,18.6,64.0,7-10,23.975128,121.61327,16.1
16,20240101091001,1,2024-01-01 09:10:00,0.09000,1017.965000,18.02,97.37,7046.501000,12.612000,2024-01-01,...,0.0,0.0,18.9,19.3,18.6,64.0,21-30,23.975128,121.61327,16.1
17,20240101092001,1,2024-01-01 09:20:00,0.01800,1018.163000,17.73,99.93,5919.584000,8.807000,2024-01-01,...,0.0,1.0,18.9,19.3,18.6,64.0,21-30,23.975128,121.61327,16.1
18,20240101093001,1,2024-01-01 09:30:00,0.01800,1018.205000,17.90,100.00,8838.918000,17.564000,2024-01-01,...,0.0,1.0,19.1,19.3,18.6,65.0,21-30,23.975128,121.61327,16.1
19,20240101094001,1,2024-01-01 09:40:00,0.03600,1018.131000,18.08,100.00,5774.665000,7.508000,2024-01-01,...,0.0,1.0,19.1,19.3,18.6,64.0,21-30,23.975128,121.61327,16.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10612,20240724162001,1,2024-07-24 16:20:00,0.00000,959.121000,25.10,100.00,758.085000,0.106000,2024-07-24,...,0.1,0.0,25.8,28.7,25.3,93.0,21-30,23.975128,121.61327,16.1
10613,20240724163001,1,2024-07-24 16:30:00,0.00000,959.102000,25.10,100.00,507.167000,0.054000,2024-07-24,...,0.1,0.0,26.9,28.7,25.3,89.0,16-20,23.975128,121.61327,16.1
10614,20240724164001,1,2024-07-24 16:40:00,0.00000,958.566667,25.10,100.00,369.998889,0.025556,2024-07-24,...,0.1,0.0,27.1,28.7,25.3,87.0,3-6,23.975128,121.61327,16.1
10615,20240724165001,1,2024-07-24 16:50:00,0.00000,958.926000,25.16,100.00,336.249000,0.020000,2024-07-24,...,0.1,0.0,26.9,28.7,25.3,88.0,3-6,23.975128,121.61327,16.1


In [39]:
os.cpu_count()

16

In [ ]:
d = {}
for i in range(1,18):
    l1 = "0" + str(i) if i<10 else str(i)
    df1 = pd.read_csv(f"./train_combine_groupby/{l1}_traingroupby.csv")
    df1 = addcolumns(df1)
    df1 = combine_weather(df1)
    df1 = selecttime(df1)
    df1.dropna(inplace=True)

    X = df1.loc[:,[ 'SunshineDuration', 'UVIndex',
                'AirTemperature', 'DailyExtremeHighAirTemperature',
                'DailyExtremeLowAirTemperature', 'RelativeHumidity',
                ]].reset_index(drop=True)
    
    Y = df1.loc[:,["Power(mW)"]].reset_index(drop=True)
    Y = Y.values.ravel()
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.01, random_state=42)

    model = RandomForestRegressor(
        n_jobs=12
    )
    scores = cross_val_score(model, X_train, y_train, scoring='neg_mean_absolute_error', cv=5)
    MAE = -sum(scores)
    model.fit(X_train, y_train)
    
    model_file_name = f'{l1}_weather.pkl'  
    model_folder_name = "train_use_full_data_update_weather_datasingle_location"         #將模型相關資訊儲存在這個檔案下
    model_file_path = os.path.join(r"D:\KPI報告\ml_power\LSTM+迴歸分析(比賽用)-20241111T013123Z-001\LSTM+迴歸分析(比賽用)\model_each_location",model_folder_name,model_file_name)
    os.makedirs(os.path.dirname(model_file_path), exist_ok=True)
    joblib.dump(model, model_file_path)
    print(model_file_name ," MAE:",MAE )
    print(f"模型已保存至: {model_file_path}")
    if model_file_name not in d:
        d[model_file_name] = MAE

01_weather.pkl  MAE: 592.6372671878407
模型已保存至: D:\KPI報告\ml_power\LSTM+迴歸分析(比賽用)-20241111T013123Z-001\LSTM+迴歸分析(比賽用)\model_each_location\train_use_full_data_update_weather_datasingle_location\01_weather.pkl
02_weather.pkl  MAE: 689.1069618931544
模型已保存至: D:\KPI報告\ml_power\LSTM+迴歸分析(比賽用)-20241111T013123Z-001\LSTM+迴歸分析(比賽用)\model_each_location\train_use_full_data_update_weather_datasingle_location\02_weather.pkl
03_weather.pkl  MAE: 752.4351211995079
模型已保存至: D:\KPI報告\ml_power\LSTM+迴歸分析(比賽用)-20241111T013123Z-001\LSTM+迴歸分析(比賽用)\model_each_location\train_use_full_data_update_weather_datasingle_location\03_weather.pkl
04_weather.pkl  MAE: 627.3513850151596
模型已保存至: D:\KPI報告\ml_power\LSTM+迴歸分析(比賽用)-20241111T013123Z-001\LSTM+迴歸分析(比賽用)\model_each_location\train_use_full_data_update_weather_datasingle_location\04_weather.pkl
05_weather.pkl  MAE: 776.0900688386986
模型已保存至: D:\KPI報告\ml_power\LSTM+迴歸分析(比賽用)-20241111T013123Z-001\LSTM+迴歸分析(比賽用)\model_each_location\train_use_full_data_update_weather_datas

# 紀錄每個model的MAE並且存成json檔案

In [ ]:
import json
json_output = json.dumps(d,indent=4)
output_path = f"/model_each_location/{model_folder_name}/d.json"
with open(output_path, "w", encoding="utf-8") as file:
    file.write(json_output)

In [ ]:
with open(f"D:\\KPI報告\\ml_power\\LSTM+迴歸分析(比賽用)-20241111T013123Z-001\\LSTM+迴歸分析(比賽用)\\model_each_location\\{model_folder_name}\\d.json", "r", encoding="utf-8") as file:
    mae_data = json.load(file)

In [22]:
#將不同觀測站的資料load近來並放在字典裡
df_dict = {}
for i in range(1,18):
    location = "0" + str(i) if i<10 else str(i)
    df = pd.read_csv(f"D:\\KPI報告\\ml_power\\LSTM+迴歸分析(比賽用)-20241111T013123Z-001\\LSTM+迴歸分析(比賽用)\\train_combine_groupby\\{location}_traingroupby.csv")
    df = addcolumns(df)
    df = combine_weather(df)
    df_dict[location] = df